In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import requests
import dash_bootstrap_components as dbc

# Sample data for the multi-bar chart
youtube_df = pd.DataFrame({
    'country': ['USA', 'Canada', 'UK', 'Australia'],
    'subscribers': [100, 200, 150, 120],
    'videos': [500, 400, 300, 200],
    'likes': [1000, 800, 600, 400],
    'comments': [300, 250, 200, 150],
    'views': [2000, 1600, 1200, 800]
})

# Sample data for the line chart (replace with your data)
data_reddit = pd.DataFrame({
    'Upload Time': pd.to_datetime(['2023-10-01 08:00:00', '2023-10-01 10:00:00', '2023-10-01 12:00:00', '2023-10-01 14:00:00', '2023-10-01 16:00:00']),
    'Views': [100, 150, 200, 250, 300],
    'Comments': [10, 15, 20, 25, 30]
})

data_source2 = pd.DataFrame({
    'Upload Time': pd.to_datetime(['2023-10-01 08:00:00', '2023-10-01 10:00:00', '2023-10-01 12:00:00', '2023-10-01 14:00:00', '2023-10-01 16:00:00']),
    'Views': [80, 120, 160, 200, 240],
    'Comments': [8, 12, 16, 20, 24]
})

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Create the multi-bar chart with percentage y-axis label
def create_multi_bar_chart():
    traces = []

    for col in ['subscribers', 'videos', 'likes', 'comments', 'views']:
        trace = go.Bar(
            x=youtube_df['country'],
            y=youtube_df[col],
            name=col,
        )
        traces.append(trace)

    layout = go.Layout(
        barmode='group',
        title='YouTube Channel Analytics by Country',
        xaxis={'title': 'Country'},
        yaxis={'title': 'Percentage (%)' if col == 'subscribers' else 'Count'},
        template='plotly_dark'
    )

    fig = go.Figure(data=traces, layout=layout)

    return dcc.Graph(figure=fig)

# Create the line chart
def create_line_chart(tab):
    if tab == 'Reddit':
        data = data_reddit
    elif tab == 'Source2':
        data = data_source2

    fig = px.line(data, x='Upload Time', y=['Views', 'Comments'], title='Views and Comments Over Time')
    fig.update_layout(
        xaxis_title='Upload Time',
        yaxis_title='Count',
        template='plotly_dark',
    )

    return dcc.Graph(figure=fig, style={'height': '90vh'})

# Define the layout of the app
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.Div([
            html.H1("Pizza Hut Social Media Data Dashboard", style={'color': 'white', 'background-color': 'black'}),
            create_multi_bar_chart(),
        ]), width=6, style={'padding': '20px', 'background-color': 'grey'}),
        dbc.Col([
            dcc.Tabs(id='tabs', value='Reddit', children=[
                dcc.Tab(label='Reddit', value='Reddit', style={'color': 'white', 'background-color': 'black', 'font-size': '20px'}),
                dcc.Tab(label='Source2', value='Source2', style={'color': 'white', 'background-color': 'black', 'font-size': '20px'}),
            ]),
            html.Div(id='tabs-content', style={'background-color': 'grey'}),
        ], width=6, style={'padding': '20px', 'background-color': 'grey'}),
    ]),
])

@app.callback(
    Output('tabs-content', 'children'),
    [Input('tabs', 'value')]
)
def render_content(tab):
    return create_line_chart(tab)

if __name__ == '__main__':
    app.run_server(debug=True)
